In [1]:
import os
import numpy as np
import pandas as pd

In [4]:
valid_df = pd.read_csv('/data/users/kyle.shaffer/dialog_data/cornell_movie_dialog_no_context_valid_retok.txt', sep='\t',
                      names=['left', 'right', 'conv_id'])
print(valid_df.shape)
valid_df.head()

(22137, 3)


,left,right,conv_id
0,"<SOD> I looked for you back at the party , but...",I was ?,conv_16
1,I was ?,"You never wanted to go out with'me , did you ?...",conv_16
2,"<SOD> Well , no",Then that's all you had to say .,conv_17
3,Then that's all you had to say .,But,conv_17
4,But,You always been this selfish ? <EOD>,conv_17


In [ ]:
def get_contexts(input_df):
    all_convos = []
    for grp_ix, grp in input_df.groupby('conv_id'):
        left, right = grp.left.tolist(), grp.right.tolist()
        unrolled_convo = []
        for lzip, rzip in zip(left, right):
            if not(lzip in unrolled_convo):
                unrolled_convo.append(lzip)
            if not(rzip in unrolled_convo):
                unrolled_convo.append(rzip)
                
        all_convos.append(unrolled_convo)
            
    return all_convos

In [ ]:
all_convos = get_contexts(valid_df)
print(len(all_convos))
all_convos[0]

In [ ]:
all_convos[7]

In [ ]:
from tqdm import tqdm

def batch_convos(convos, n=3):
    batched_convos = []
    pad = '<PAD>'
    for c in tqdm(convos):
        if len(c) < n:
            diff = n - len(c)
            max_len = max([len(u.split()) for u in c])
            padded_sent = [pad] * max_len
            for _ in range(diff):
                c.insert(0, padded_sent)
            batched_convos.append(c)
        elif len(c) == 3:
            batched_convos.append(c)
        else:
            for u_ix in range(len(c) - (n - 1)):
                batched_convos.append(c[u_ix: (u_ix + n)])
                
    return batched_convos

In [ ]:
batched_convos = batch_convos(all_convos)
len(batched_convos)

In [ ]:
batched_convos[0]

In [ ]:
batched_convos[1]

In [ ]:
batched_convos[7]

In [ ]:
batched_convos[8]

In [ ]:
batched_convos[9]

In [ ]:
batched_convos[10]

In [ ]:
batched_convos[28]

In [ ]:
all_convos[7]

In [ ]:
with open('/data/users/kyle.shaffer/dialog_data/cornell_movie_context_train.txt', mode='w') as outfile:
    for bc in batched_convos:
        if isinstance(bc[0], str):
            outfile.write(bc[0])
        else:
            outfile.write(' '.join(bc[0]))
        outfile.write('\t')
        outfile.write(bc[1])
        outfile.write('\t')
        outfile.write(bc[2])
        outfile.write('\n')

In [5]:
# Double-checking and writing out new vocab
from collections import Counter

# vocab_cnt = Counter()

for l, r in zip(valid_df.left.tolist(), valid_df.right.tolist()):
    vocab_cnt.update(l.split())
    vocab_cnt.update(r.split())
    
print('Vocab Size:', len(vocab_cnt))
vocab_cnt.most_common(10)

Vocab Size: 80368


[('.', 491821),
 (',', 241654),
 ('?', 163707),
 ('I', 151433),
 ('you', 144522),
 ('the', 124995),
 ('to', 114282),
 ('a', 95591),
 ('<SOD>', 83033),
 ('<EOD>', 83033)]

In [6]:
with open('/data/users/kyle.shaffer/dialog_data/cornell_movie_vocab.txt', mode='w') as outfile:
    for w, c in vocab_cnt.most_common():
        outfile.write(w)
        outfile.write('\t')
        outfile.write(str(c))
        outfile.write('\n')